# **Algorithmic Methods of Data Mining - Fall 2022**

## **Homework 4: Getting to know your customers**

**Packages that are used troughout the notebook:**

In [ ]:
import pandas as pd

### 1. Finding Similar Costumers

#### 1.1 Set up the data

In [ ]:
data = pd.read_csv('bank_transactions.csv')
data.dropna(inplace=True)

#### 1.2 Fingerprint hashing

#### 1.3 Locality Sensitive Hashing

### 2. Grouping customers together!

#### 2.1 Set up the data

#### 2.2 Choose your features (variables)!

#### 2.3 Clustering!

#### 2.4 Analysing your results!

### Bonus

### Command Line Question

### Algorithmic Question